In [2]:
import splitting as sp
import train
from pruning_utils.prune import *
from pruning_utils.pruners import *
from pruning_utils.generator import *

# Magnitude PaI

In [10]:
trainLoader, testLoader = train.mnist_loaders(train_batch_size=128, test_batch_size=400)

model = train.SingleFcNet(sp.MONPeacemanRachford,
                        in_dim=28**2,
                        out_dim=87,
                        alpha=1.0,
                        max_iter=300,
                        tol=1e-2,
                        m=1.0,
                        is_pruning=True)

# print('Pruning with {} for {} epochs.'.format(args.pruner, args.prune_epochs))
### Pruning
masked_parameters_ = masked_parameters(model)
pruner = Mag(masked_parameters_)

###################################################
####### Change compression to change the sparsity # 
###################################################
compression = 1

sparsity = 10**(-float(compression))
print(f'sparsity is {1 - sparsity}')
prune_loop(model, None, pruner, None, 'cpu', sparsity, schedule='exponential', scope='global', epochs=1,
               reinitialize=False, train_mode=False, shuffle=False, invert=False,)

### Training
train.train(trainLoader, testLoader,
            model,
            max_lr=1e-3,
            lr_mode='step',
            step=10,
            change_mo=False,
            # epochs=40,
            epochs=1,
            print_freq=100,
            tune_alpha=True)


sparsity is 0.9


100%|██████████| 1/1 [00:00<00:00, 333.25it/s]


----tuning alpha----
current:  1.0
alpha: 1.0	 iters: 6
alpha: 0.5	 iters: 6
alpha: 0.25	 iters: 8
setting to:  0.5
--------------

Train Epoch: 0.21 [12928/60000 (21%)]	Loss: 2.2957	Error: 87.50
Fwd iters: 6.00	Fwd Time: 0.0058	Bkwd Iters: 6.00	Bkwd Time: 0.0051

Train Epoch: 0.43 [25728/60000 (43%)]	Loss: 2.2961	Error: 86.72
Fwd iters: 6.00	Fwd Time: 0.0059	Bkwd Iters: 6.00	Bkwd Time: 0.0051

----tuning alpha----
current:  0.5
alpha: 1.0	 iters: 6
alpha: 0.5	 iters: 6
alpha: 0.25	 iters: 9
setting to:  0.5
--------------

Train Epoch: 0.64 [38528/60000 (64%)]	Loss: 2.3004	Error: 85.94
Fwd iters: 6.00	Fwd Time: 0.0059	Bkwd Iters: 6.00	Bkwd Time: 0.0051

Train Epoch: 0.85 [51328/60000 (85%)]	Loss: 2.2994	Error: 89.84
Fwd iters: 6.00	Fwd Time: 0.0060	Bkwd Iters: 6.00	Bkwd Time: 0.0050

Tot train time: 13.237984418869019


Test set: Average loss: 2.3013, Error: 8865/10000 (88.65%)
Tot test time: 1.2776570320129395






Model collapse because all parameters in the first layer are pruned

In [11]:
for name, mask in model.named_buffers():
    n_kept = mask.sum()
    n_params = mask.numel()
    
    print('n_kept / total params {}/{} ~ ({:.4f})'.format(n_kept, n_params, n_kept/n_params*100))

n_kept / total params 0.0/68208 ~ (0.0000)
n_kept / total params 87.0/87 ~ (100.0000)
n_kept / total params 4184.0/7569 ~ (55.2781)
n_kept / total params 4151.0/7569 ~ (54.8421)


In [7]:
model

SingleFcNet(
  (mon): MONPeacemanRachford(
    (linear_module): MaskedMONSingleFc(
      (U): Linear(in_features=784, out_features=87, bias=True)
      (A): Linear(in_features=87, out_features=87, bias=False)
      (B): Linear(in_features=87, out_features=87, bias=False)
    )
    (nonlin_module): MONReLU()
  )
  (Wout): Linear(in_features=87, out_features=10, bias=True)
)

# Random PaI

In [13]:
trainLoader, testLoader = train.mnist_loaders(train_batch_size=128, test_batch_size=400)

model = train.SingleFcNet(sp.MONPeacemanRachford,
                        in_dim=28**2,
                        out_dim=87,
                        alpha=1.0,
                        max_iter=300,
                        tol=1e-2,
                        m=1.0,
                        is_pruning=True)

# print('Pruning with {} for {} epochs.'.format(args.pruner, args.prune_epochs))
### Pruning
masked_parameters_ = masked_parameters(model)
pruner = Rand(masked_parameters_)

###################################################
####### Change compression to change the sparsity # 
###################################################
compression = 3     

sparsity = 10**(-float(compression))
print(f'sparsity is {1 - sparsity}')
prune_loop(model, None, pruner, None, 'cpu', sparsity, schedule='exponential', scope='global', epochs=1,
               reinitialize=False, train_mode=False, shuffle=False, invert=False,)

### Training
train.train(trainLoader, testLoader,
            model,
            max_lr=1e-3,
            lr_mode='step',
            step=10,
            change_mo=False,
            # epochs=40,
            epochs=1,
            print_freq=100,
            tune_alpha=True)


sparsity is 0.999


100%|██████████| 1/1 [00:00<00:00, 499.98it/s]


----tuning alpha----
current:  1.0
alpha: 1.0	 iters: 2
alpha: 0.5	 iters: 5
setting to:  1.0
--------------

Train Epoch: 0.21 [12928/60000 (21%)]	Loss: 2.1763	Error: 51.56
Fwd iters: 2.00	Fwd Time: 0.0047	Bkwd Iters: 3.00	Bkwd Time: 0.0029

Train Epoch: 0.43 [25728/60000 (43%)]	Loss: 1.7987	Error: 46.09
Fwd iters: 2.00	Fwd Time: 0.0046	Bkwd Iters: 2.58	Bkwd Time: 0.0025

----tuning alpha----
current:  1.0
alpha: 1.0	 iters: 2
alpha: 0.5	 iters: 5
setting to:  1.0
--------------

Train Epoch: 0.64 [38528/60000 (64%)]	Loss: 1.4094	Error: 36.72
Fwd iters: 2.00	Fwd Time: 0.0047	Bkwd Iters: 2.00	Bkwd Time: 0.0021

Train Epoch: 0.85 [51328/60000 (85%)]	Loss: 1.2550	Error: 36.72
Fwd iters: 2.00	Fwd Time: 0.0048	Bkwd Iters: 2.00	Bkwd Time: 0.0020

Tot train time: 11.461966753005981


Test set: Average loss: 1.1365, Error: 3425/10000 (34.25%)
Tot test time: 1.250028133392334






In [14]:
for name, mask in model.named_buffers():
    n_kept = mask.sum()
    n_params = mask.numel()
    
    print('n_kept / total params {}/{} ~ ({:.4f})'.format(n_kept, n_params, n_kept/n_params*100))

n_kept / total params 76.0/68208 ~ (0.1114)
n_kept / total params 87.0/87 ~ (100.0000)
n_kept / total params 1.0/7569 ~ (0.0132)
n_kept / total params 7.0/7569 ~ (0.0925)


In [1]:
import torch

In [2]:
a = torch.tensor([1,2,2,2,2,2,33,3])
a 

tensor([ 1,  2,  2,  2,  2,  2, 33,  3])

In [3]:
torch.topk(a,3, largest=True)

torch.return_types.topk(
values=tensor([33,  3,  2]),
indices=tensor([6, 7, 1]))

In [4]:
torch.kthvalue(a, 3)

torch.return_types.kthvalue(
values=tensor(2),
indices=tensor(5))

In [7]:
1 - 10**(-0.25)

0.4376586748096509